## Import Libraries

In [41]:
import matplotlib.pyplot as plt
from numpy import sin, cos, sqrt, pi, linspace, arange, deg2rad, rad2deg, array, arcsin, arccos,sort, argsort, argwhere, argmin, argmax, interp, concatenate, zeros
from scipy.spatial import distance
from numpy import linalg as LA
import warnings
warnings.simplefilter('ignore')

def sec(x):
    return 1/cos(x)
def tan(x):
    return sin(x)/cos(x)

## Initialize Parameters 

In [45]:
n      = 1.33
c      = 299792458/n
h      = 4.5
Dia    = 7.3   #--------------------- diameter of tank
R      = 7.3/2 #--------------------- radius of tank
v      = n * c
times  = linspace(1e-11,1e-6,200000)

theta  = deg2rad(180)   #----------------------------------------- Zenith  varies from 90 to 180                    #------PARAM
phi    = deg2rad(0)   #----------------------------------------- Azimuth varies from  0 to 360                    #------PARAM

x1,y1,z1 = [0.001,0.0,h] #----------------------------------------- Coordinates of particle at top tanker lid      #------PARAM
x2,y2,z2 = [x1+h*tan(theta)*cos(phi), y1+h*tan(theta)*sin(phi), 0] #-------------------- Calculate bottom y coordinate

A      = array([x1,y1,z1])
B      = array([x2,y2,0 ])
D1     = array([0, 0, 0])                                #------ Central    PMT number 1 
D2     = array([1.85*cos(2*pi/3) , 1.85*sin(2*pi/3), 0]) #------ Non-Radial PMT number 2
D3     = array([1.85*cos(4*pi/3) , 1.85*sin(4*pi/3), 0]) #------ Non-Radial PMT number 3
D4     = array([1.85*cos(0)      , 1.85*sin(0     ), 0]) #------ Radial     PMT number 4

nAB    = LA.norm(B-A)
nBD1   = LA.norm(B-D1)
nBD2   = LA.norm(B-D2)
nBD3   = LA.norm(B-D3)
nBD4   = LA.norm(B-D4)

theta2    = pi - theta
del theta
theta     = theta2

limit     = h 
print('A   : ', A)
print('B   : ', B)
print('D1  : ', D1)
print('D2  : ', D2)
print('D3  : ', D3)
print('D4  : ', D4)
print('AB  : ', nAB)
print('BD1 :  %.2f m\nBD2 :  %.2f m\nBD3 :  %.2f m\nBD4 :  %.2f m'%(nBD1,nBD2,nBD3,nBD4) )
print('Upper limit: ', limit, "m")
print('Theta: ',theta)

A   :  [1.0e-03 0.0e+00 4.5e+00]
B   :  [0.001 0.    0.   ]
D1  :  [0 0 0]
D2  :  [-0.925     1.602147  0.      ]
D3  :  [-0.925    -1.602147  0.      ]
D4  :  [1.85 0.   0.  ]
AB  :  4.5
BD1 :  0.00 m
BD2 :  1.85 m
BD3 :  1.85 m
BD4 :  1.85 m
Upper limit:  4.5 m
Theta:  0.0


## Computations 

In [46]:
def entry_brightness(h,c,v,theta,L,ang1,nAB,den,A,B,D):
    x         = 0
    t1        = 0
    k         = sqrt( L**2 + (nAB-x)**2 - 2*L*(nAB-x)*cos(ang1) )
    t2        = k/c
    tt        = t1 + t2
    xtermA    = ( c*c*tt*v - v*v*nAB + L*v*v*cos(ang1) )
    xtermB    = (-v*v*L*L) + ( c*c*(L*L +(nAB-tt*v)**2) ) - ( 2*c*c*L*(nAB-tt*v)*cos(ang1) ) + ( (L*v*cos(ang1))**2 )
    vtermA    = v*v*(tt*v - nAB + L*cos(ang1))
    xp, xm    = (xtermA + sqrt(v*v*xtermB))/den , (xtermA - sqrt(v*v*xtermB))/den
    zp, zm    = nAB-xp , nAB-xm
    vp, vm    = ((c*c*v) * ( 1 + (vtermA/sqrt(v*v*xtermB)) ))/den , ((c*c*v) * ( 1 - (vtermA/sqrt(v*v*xtermB)) ))/den
    r, rp, rm = x/nAB, xp/nAB, xm/nAB
    X        = array([ (1-r) *A[0] + r *B[0] , (1-r) *A[1] + r *B[1] , (1-r) *A[2]+r *B[2] ])
    Xp       = array([ (1-rp)*A[0] + rp*B[0] , (1-rp)*A[1] + rp*B[1] , (1-rp)*A[2]+rp*B[2] ])
    Xm       = array([ (1-rm)*A[0] + rm*B[0] , (1-rm)*A[1] + rm*B[1] , (1-rm)*A[2]+rm*B[2] ])
    kp, km   = LA.norm(D-Xp) , LA.norm(D-Xm)
    phip     = arccos (   sum( (D-Xp)*(D-B) ) / (L*LA.norm(D-Xp))   )
    phim     = arccos (   sum( (D-Xm)*(D-B) ) / (L*LA.norm(D-Xm))   )
    alphap   = arccos (   sum( (D-Xp)*(B-Xp)) / (LA.norm(B-Xp)*LA.norm(D-Xp)) )
    alpham   = arccos (   sum( (D-Xm)*(B-Xm)) / (LA.norm(B-Xm)*LA.norm(D-Xm)) )
    vtp      = vp * sin(alphap)
    vtm      = vm * sin(alpham)
    omegap   = (vtp / kp)
    omegam   = (vtm / km)
    bp       = abs(omegap / (kp**2))
    bm       = abs(omegam / (km**2))
    return bp

def plus_phi_vs_t(tp,phip,color,mylabel,phi):
    plt.plot(tp, phip, c=color, ls='-' , label=mylabel, lw=2)
    if detector!=4:
        plt.axhline(phi, c='k', ls='--')
        if detector==2:
            plt.text(28, phi-1, r'$\theta_C = %.2f ^{\circ}$'%phi , fontsize=15)
        plt.text(28, phi+1, r'$\theta_C = %.2f ^{\circ}$'%phi , fontsize=15)       
    plt.axvline(0, c='k', ls=':')
    
def minus_phi_vs_t(tm,phim,color,mylabel,phi):
    plt.plot(tm, phim, c=color, ls='-.' , label=mylabel,lw=2)
    if detector!=4:
        plt.axhline(phi, c='k', ls='--')
        if detector==2:
            plt.text(28, phi-1, r'$\theta_C = %.2f ^{\circ}$'%phi , fontsize=15)
        plt.text(28, phi+1, r'$\theta_C = %.2f ^{\circ}$'%phi , fontsize=15)
    plt.axvline(0, c='k', ls=':')

def both_phi_vs_t(tp,phip,tm,phim,color,mylabel,phi):
    plt.plot(tp, phip, c=color, ls='-' , label=mylabel,lw=2)
    plt.plot(tm, phim, c=color, ls='-.',lw=2)
    if detector!=4:
        plt.axhline(phi, c='k', ls='--')
        if detector==2:
            plt.text(28, phi-4, r'$\theta_C = %.2f ^{\circ}$'%phi , fontsize=15)
        else:
            plt.text(28, phi+1, r'$\theta_C = %.2f ^{\circ}$'%phi , fontsize=15)
    
    plt.axvline(0, c='k', ls=':')
    

def plus_b_vs_t(t,bp,color,mylabel):  # Use times  = linspace(1e-12,1e-7,2000000)
    plt.plot(t, bp, c=color, ls='-', label=mylabel, lw=2.5)
    plt.axhline(1, c='k', ls=':')
    plt.axvline(0, c='k', ls=':')

def minus_b_vs_t(t,bm,color,mylabel):  # Use times  = linspace(1e-12,1e-7,2000000)
    plt.plot(t, bm, c=color, ls='--',label=mylabel, lw=2.5)
    plt.axhline(1, c='k', ls=':')
    plt.axvline(0, c='k', ls=':')

def both_b_vs_t(tp,bp,tm,bm,color,mylabel):  # Use times  = linspace(1e-12,1e-7,2000000)
    plt.plot(tp, bp, c=color, ls='-', label=mylabel, lw=2.5)
    plt.plot(tm, bm, c=color, ls='--', lw=2.5)
    plt.axhline(1, c='k', ls=':')
    plt.axvline(0, c='k', ls=':')

    
def plus_b_vs_phi(phip,bp,color,mylabel,phi):
    plt.plot(phip, bp, c=color, ls='-', label=mylabel, lw=2.5)
    plt.axhline(1,  c='k', ls=':')

def minus_b_vs_phi(phim,bm,color,mylabel,phi):
    plt.plot(phim, bm, c=color, ls='--',label=mylabel, lw=2.5)
    plt.axhline(1,  c='k', ls=':')

def both_b_vs_phi(phip,bp,phim,bm,color,mylabel,phi):
    plt.plot(phip, bp, c=color, ls='-', label=mylabel, lw=2.5)
    plt.plot(phim, bm, c=color, ls='--', lw=2.5)
    if detector!=4:
        plt.axvline(phi,c='k', ls=':')
        if detector==3:
            plt.text(phi+0.5, 500, r'$\theta_C = %.2f ^{\circ}$'%phi , fontsize=15)
        elif detector==1:
            plt.text(phi+0.5, 1.2, r'$\theta_C = %.2f ^{\circ}$'%phi , fontsize=15)
        else:
            plt.text(phi-10, 2000, r'$\theta_C = %.2f ^{\circ}$'%phi , fontsize=15)
    plt.axhline(1, c='k', ls=':')
    
    
def plus_z_vs_t(tp,zp,color,mylabel,zc):
    plt.plot(tp, zp, c=color, ls='-' , label=mylabel, lw=2.5)
    if (0<=zc<=limit):
        plt.text(30, zc+0.05, r'$z_C = %.2f $'%(round(zc,2)) , fontsize=15)
        plt.axhline(zc, c='k', ls=':')
    plt.axvline(0, c='k', ls=':')

def minus_z_vs_t(tm,zm,color,mylabel,zc):
    plt.plot(tm, zm, c=color, ls='-.' , label=mylabel, lw=2.5)
    if (0<=zc<=limit):
        plt.text(30, zc+0.05, r'$z_C = %.2f $'%(round(zc,2)) , fontsize=15)
        plt.axhline(zc, c='k', ls=':')
    plt.axvline(0, c='k', ls=':')

def both_z_vs_t(tp,zp,tm,zm,color,mylabel,zc):
    plt.plot(tp, zp, c=color, ls='-' , label=mylabel, lw=2.5)
    plt.plot(tm, zm, c=color, ls='-.', lw=2.5)
    if (0<=zc<=limit):
        if detector==3:
            plt.text(12, 2.1, r'$z_C = %.2f $'%(round(zc,2)) , fontsize=15)
        elif detector==1:
            plt.text(13, zc+0.05, r'$z_C = %.2f $'%(round(zc,2)) , fontsize=15)     
        else:
            plt.text(30, zc+0.05, r'$z_C = %.2f $'%(round(zc,2)) , fontsize=15)
        plt.axhline(zc, c='k', ls=':')
    plt.axvline(0, c='k', ls=':')


In [44]:
%matplotlib
plt.figure(figsize=(15,12))

#plotme      = 'b vs phi'
plotme      = 'b vs t'
#plotme      = 'phi vs t'
#plotme      = 'z vs t'

detectors   = [1,4]
distances   = [nBD1,nBD4]
coordinates = [D1,D4]
colors      = ['k','r']
labels      = ['Central Detector', 'Surrounding Detectors']

for detector, distance, coordinate, color, label in zip(detectors, distances, coordinates, colors, labels):
    
    print(":::::::::::::::::::::::::::::::\n","Detector ", detector)
    
    L      = distance
    nBD    = distance
    D      = coordinate
    ang1   = sum((D-B)*(A-B)) / (nBD*nAB)
    den    = (c**2-v**2)
    xc_num = nAB*den - L*den*cos(ang1) + sqrt( c*c*L*L*den*(cos(ang1)**2-1)  )
    xc     = xc_num/den
    zc     = (nAB - xc) * cos(theta)
    print('zc: ', round(zc,3))
    
    T, XP, XM, ZP, ZM, BP, BM, PHIP, PHIM = [], [], [], [], [], [], [], [], []

    for iii,t in enumerate(times):
        
        x         = v*t
        t1        = t
        k         = sqrt( L**2 + (nAB-x)**2 - 2*L*(nAB-x)*cos(ang1) )
        t2        = k/c
        tt        = t1 + t2
        xtermA    = ( c*c*tt*v - v*v*nAB + L*v*v*cos(ang1) )
        xtermB    = (-v*v*L*L) + ( c*c*(L*L +(nAB-tt*v)**2) ) - ( 2*c*c*L*(nAB-tt*v)*cos(ang1) ) + ( (L*v*cos(ang1))**2 )
        vtermA    = v*v*(tt*v - nAB + L*cos(ang1))
        
        xp, xm    = (xtermA + sqrt(v*v*xtermB))/den , (xtermA - sqrt(v*v*xtermB))/den
        zp, zm    = nAB-xm , nAB-xp
        vp, vm    = ((c*c*v) * ( 1 + (vtermA/sqrt(v*v*xtermB)) ))/den , ((c*c*v) * ( 1 - (vtermA/sqrt(v*v*xtermB)) ))/den
        
        r, rp, rm = x/nAB, xp/nAB, xm/nAB
        
        X        = array([ (1-r) *A[0] + r *B[0] , (1-r) *A[1] + r *B[1] , (1-r) *A[2]+r *B[2] ])
        Xp       = array([ (1-rp)*A[0] + rp*B[0] , (1-rp)*A[1] + rp*B[1] , (1-rp)*A[2]+rp*B[2] ])
        Xm       = array([ (1-rm)*A[0] + rm*B[0] , (1-rm)*A[1] + rm*B[1] , (1-rm)*A[2]+rm*B[2] ])
        
        kp, km   = LA.norm(D-Xp) , LA.norm(D-Xm)
        
        phip     = arccos (   sum( (D-Xp)*(D-B) ) / (L*LA.norm(D-Xp))   )
        phim     = arccos (   sum( (D-Xm)*(D-B) ) / (L*LA.norm(D-Xm))   )
        
        alphap   = arccos (   sum( (D-Xp)*(B-Xp)) / (LA.norm(B-Xp)*LA.norm(D-Xp)) )
        alpham   = arccos (   sum( (D-Xm)*(B-Xm)) / (LA.norm(B-Xm)*LA.norm(D-Xm)) )
        
        vtp      = vp * sin(alphap)
        vtm      = vm * sin(alpham)
        omegap   = (vtp / kp)
        omegam   = (vtm / km)
        bp       = abs(omegap / (kp**2))
        bm       = abs(omegam / (km**2))
        
        phip, phim = rad2deg(phip), rad2deg(phim)
        
        XP  .append(xp*cos(theta))
        XM  .append(xm*cos(theta))
        ZP  .append(zp*cos(theta))
        ZM  .append(zm*cos(theta))
        BP  .append(bp)
        BM  .append(bm)            
        PHIP.append(phip)
        PHIM.append(phim)
        T   .append(tt*1e9)
        
        #if (detector==2) and (iii%10000==0):
        #    print(tt,zp,zm,alphap,alpham,phip,phim,bp,bm,sep="  ")
        
    r4  = xc/nAB
    Xc  = array([ (1-r4)*A[0] + r4*B[0] , (1-r4)*A[1] + r4*B[1] , (1-r4)*A[2]+r4*B[2] ])
    PHI = arccos( sum((D-Xc)*(D-B))  / (L*LA.norm(D-Xc)) ) ; PHI = rad2deg(PHI)

    #print("PHI: ",round(PHI,2), "ZC: ", round(zc,2))
    XP, XM, ZP, ZM, TT, BP, BM, PHIP, PHIM = array(XP), array(XM), array(ZP), array(ZM), array(T), array(BP), array(BM), array(PHIP), array(PHIM)

    conp     = [(ZP>=0) & (ZP<=limit)]
    conm     = [(ZM>=0) & (ZM<=limit)]
    pluslen  = len (ZP[conp])
    minuslen = len (ZM[conm])
    
    if detector==1:
        den_bright = entry_brightness(h=h,c=c,v=v,theta=theta,L=L,ang1=ang1,nAB=nAB,den=den,A=A,B=B,D=D)
    
    if pluslen == 0:
        if minuslen == 0:
            print("Images outside tank. Skipping...")
        else:
            print('Only image moving down. Processing...')
            TTm,ZM,BM,PHIM = TT[conm],ZM[conm],BM[conm]/den_bright,PHIM[conm]
            if plotme=='b vs phi':
                minus_b_vs_phi(phim=PHIM,bm=BM,color=color,mylabel=label,phi=PHI)
            elif plotme=='b vs t':
                minus_b_vs_t(t=TTm,bm=BM,color=color,mylabel=label)
            elif plotme=='phi vs t':
                minus_phi_vs_t(tm=TTm,phim=PHIM,color=color,mylabel=label,phi=PHI)
            elif plotme=='z vs t':
                minus_z_vs_t(tm=TTm,zm=ZM,color=color,mylabel=label,zc=zc)
    elif pluslen != 0:
        if minuslen == 0:
            print('Only image moving up. Processing...')
            TTp,ZP,BP,PHIP = TT[conp],ZP[conp],BP[conp]/den_bright,PHIP[conp]
            print(BP[:10], BM[:10])
            if plotme=='b vs z':
                plus_b_vs_phi(phip=PHIP,bp=BP,color=color,mylabel=label,phi=PHI)
            elif plotme=='b vs t':
                plus_b_vs_t(t=TTp,bp=BP,color=color,mylabel=label)
            elif plotme=='phi vs t':
                plus_phi_vs_t(tp=TTp,phip=PHIP,color=color,mylabel=label,phi=48)
            elif plotme=='z vs t':
                plus_z_vs_t(tp=TTp,zp=ZP,color=color,mylabel=label,zc=zc)
        else:
            print('Both images moving. Processing...')
            TTp,TTm,ZP,ZM,BP,BM,PHIP,PHIM = TT[conp],TT[conm],ZP[conp],ZM[conm],BP[conp]/den_bright,BM[conm]/den_bright,PHIP[conp],PHIM[conm]
            print(BP[:10], BM[:10])
            if plotme=='b vs phi':
                both_b_vs_phi(phip=PHIP,bp=BP,phim=PHIM,bm=BM,color=color,mylabel=label,phi=PHI)
            elif plotme=='b vs t':
                both_b_vs_t(tp=TTp,bp=BP,tm=TTm,bm=BM,color=color,mylabel=label)
            elif plotme=='phi vs t':
                both_phi_vs_t(tp=TTp,phip=PHIP,tm=TTm,phim=PHIM,color=color,mylabel=label,phi=PHI)
            elif plotme=='z vs t':
                both_z_vs_t(tp=TTp,zp=ZP,tm=TTm,zm=ZM,color=color,mylabel=label,zc=zc)

plt.legend(loc='upper right', prop={'size': 14})
plt.tick_params(axis='both', direction='in', labelsize=18)

plt.yscale('log')
plt.show()

Using matplotlib backend: Qt5Agg
:::::::::::::::::::::::::::::::
 Detector  1
zc:  0.001
Both images moving. Processing...
[1.06873242e+11 4.81652444e+11 5.18118305e+12 3.28486403e+11] [354.61270189 355.40274763 356.19499474 356.98945065 357.78612274
 358.5850184  359.38614513 360.18951037 360.99512169 361.80298655]
:::::::::::::::::::::::::::::::
 Detector  4
zc:  1.849
Both images moving. Processing...
[1353.73454459 1356.30961184 1358.89059727 1361.47751661 1364.07038561
 1366.66922009 1369.2740359  1371.88484895 1374.50167518 1377.12453059] [3436.25006045 3434.96827176 3433.68689266 3432.40592311 3431.1253631
 3429.84521262 3428.56547164 3427.28614015 3426.00721812 3424.72870553]


In [38]:
TTp.min() , TTm.min()

(8.84336739503298, 8.84336739503298)

In [40]:
plt.plot(TTm,BM/den_bright)
plt.plot(TTp,BP/den_bright)

In [35]:
TTm[:20],BM[:20]

(array([11.76057294, 11.75397293, 11.74737291, 11.7407729 , 11.73417289,
        11.72757288, 11.72097286, 11.71437285, 11.70777284, 11.70117282,
        11.69457281, 11.6879728 , 11.68137279, 11.67477277, 11.66817276,
        11.66157275, 11.65497274, 11.64837272, 11.64177271, 11.6351727 ]),
 array([3432.21967727, 3429.15072113, 3426.08411483, 3423.01985813,
        3419.95795076, 3416.89839245, 3413.84118294, 3410.78632195,
        3407.73380921, 3404.68364442, 3401.63582731, 3398.59035757,
        3395.5472349 , 3392.50645902, 3389.46802959, 3386.43194633,
        3383.3982089 , 3380.36681698, 3377.33777025, 3374.31106838]))

In [36]:
TTp[:20], BP[:20]

(array([11.76057294, 11.75397293, 11.74737291, 11.7407729 , 11.73417289,
        11.72757288, 11.72097286, 11.71437285, 11.70777284, 11.70117282,
        11.69457281, 11.6879728 , 11.68137279, 11.67477277, 11.66817276,
        11.66157275, 11.65497274, 11.64837272, 11.64177271, 11.6351727 ]),
 array([1354.1976818 , 1360.38845747, 1366.61344253, 1372.87285548,
        1379.16691637, 1385.49584684, 1391.85987007, 1398.25921089,
        1404.69409569, 1411.1647525 , 1417.67141099, 1424.21430245,
        1430.79365986, 1437.40971785, 1444.06271273, 1450.75288251,
        1457.48046692, 1464.24570741, 1471.04884715, 1477.89013107]))

In [28]:
kk=[print(i) if i=='False' for i in in TTp[:100] == TTm[:100]]

SyntaxError: invalid syntax (<ipython-input-28-ac7ff3dbbca6>, line 1)

In [ ]:
# phi vs t
plt.xlabel('time since muon entry (ns)',fontsize=18)
plt.ylabel(r'angular locations $\phi_{pm}\;(in\;degrees)$', fontsize=18)
plt.xticks(arange(10,37,2))
plt.xlim(10.02,36.07)

# b vs t
plt.xlabel('time since muon entry (ns)',fontsize=18)
plt.ylabel('relative brightness ($b/b_{entry}$)', fontsize=18) 
plt.xticks(arange(11,38,2))
plt.yscale('log')
plt.xlim(10.02,36.07)
plt.ylim(0.34,286.5)

# b vs phi
plt.xlabel(r'angular locations $\phi_{pm}\;(in\;degrees)$',fontsize=18)
plt.ylabel(r'relative brightness ($b/b_{entry}$)', fontsize=18) 
plt.yscale('log')

# z vs t
plt.xlabel('time since muon entry (ns)',fontsize=18)
plt.ylabel(r'image heights $z_{pm} (in\;meters)$'     , fontsize=18).set_rotation(90)
plt.xticks(arange(10,38,2))
plt.ylim(-0.02,limit)
plt.xlim(9.99,36.07)